### Loading environment

In [1]:
%load_ext autoreload
%autoreload 2

from aiida import orm, engine, load_profile
from aiida.common import NotExistent

# import aiida_dislocation

# import importlib
# importlib.reload(aiida_dislocation.workflows.usf)
from aiida_dislocation.workflows.usf import USFWorkChain

from ase.io import read
from ase.build import bulk
from aiida.engine import run

import yaml

import logging


load_profile()

Profile<uuid='155a05c5c8f7420aa49bbf9207908698' name='supercon'>

In [2]:
prepend_text = """
module load intel
module load mkl
module load impi
module load fftw/3.3.10
"""

group_label = 'test/workchains/USF/'
test_USF_group, created = orm.Group.collection.get_or_create(label=group_label)
if created:
    print(f"Group '{group_label}' created.")
else:
    print(f"Group '{group_label}' already exists.")


computer = 'mn5'
mn5_computer = orm.load_computer(computer)
mn5_computer.set_workdir('/gpfs/scratch/ehpc14/uclo653525/aiida_workdir/test_aiida_dislocation')

version = '730'
pw_label = f'qe-{version}-pw'
pw_code = orm.load_code(f'{pw_label}@{computer}')


Group 'test/workchains/USF/' already exists.


## TEST A1 -- Aluminium

In [3]:
import yaml

ase_atom = read('./structures/cif/Al-225-A1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
# inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5


### {100} surface

In [9]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '100', [0.5, 0.5]])
inputs['clean_workdir'] = orm.Bool(False)
logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

extras = {
    'kpoints': kpoints.get_kpoints_mesh()[0],
    'degauss': inputs['pw']['parameters']['SYSTEM']['degauss'],
    'formula': inputs['structure'].get_formula(),
    'structure_type': inputs['slipping_system'][0],
    'gliding_plane': inputs['slipping_system'][1],
    'slipping_direction': inputs['slipping_system'][2]
}

# wc = engine.submit(USFWorkChain, **inputs)
# test_USF_group.add_nodes(wc)
# wc.base.extras.set_many(extras=extras)


05/15/2025 10:48:30 AM <420539> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140158|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/15/2025 10:48:30 AM <420539> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140158|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
05/15/2025 10:48:30 AM <420539> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140158|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. 
05/15/2025 10:48:30 AM <420539> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140158|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.5]. 
05/15/2025 10:48:30 AM <420539> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh.Migh

### {110} surface

In [4]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['slipping_system'] = orm.List(['A1', '110', [0.5, 0.5]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/15/2025 12:02:06 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140324|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/15/2025 12:02:06 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140324|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
05/15/2025 12:02:06 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140324|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. 
05/15/2025 12:02:06 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140324|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.5]. 
05/15/2025 12:02:06 PM <643100> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh.Migh

### {111} surface

In [5]:
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(9)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['slipping_system'] = orm.List(['A1', '111', [1/2, 0]])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/15/2025 11:38:39 AM <542299> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140274|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/15/2025 11:38:39 AM <542299> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140274|USFWorkChain|validate_slipping_system]: Now we are working on 111 gliding plane.
05/15/2025 11:38:39 AM <542299> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140274|USFWorkChain|validate_slipping_system]: Faulting is possible for 111 gliding plane.The stacking order is ABC. 
05/15/2025 11:38:39 AM <542299> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140274|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0]. 
05/15/2025 11:38:39 AM <542299> aiida.workflow.dislocation: [WARNING] Supercell is not integer multiple of unit cell. Ceiling the kpoints mesh.Might cause ene

## TEST A1 -- Copper

In [8]:
import yaml

ase_atom = read('./structures/cif/Cu-225-A1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
# inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5


### {100} surface

In [11]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '100', [0.5, 0.0]])
inputs['clean_workdir'] = orm.Bool(False)
logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

extras = {
    'kpoints': kpoints.get_kpoints_mesh()[0],
    'degauss': inputs['pw']['parameters']['SYSTEM']['degauss'],
    'formula': inputs['structure'].get_formula(),
    'structure_type': inputs['slipping_system'][0],
    'gliding_plane': inputs['slipping_system'][1],
    'slipping_direction': inputs['slipping_system'][2]
}

# wc = engine.submit(USFWorkChain, **inputs)
# test_USF_group.add_nodes(wc)
# wc.base.extras.set_many(extras=extras)


05/15/2025 03:57:24 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140489|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/15/2025 03:57:24 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140489|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
05/15/2025 03:57:24 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140489|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. 
05/15/2025 03:57:24 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140489|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.0]. 
05/15/2025 03:57:24 PM <1288291> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh

In [12]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A1', '110', [0.5, 0.0]])
inputs['clean_workdir'] = orm.Bool(False)
logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

extras = {
    'kpoints': kpoints.get_kpoints_mesh()[0],
    'degauss': inputs['pw']['parameters']['SYSTEM']['degauss'],
    'formula': inputs['structure'].get_formula(),
    'structure_type': inputs['slipping_system'][0],
    'gliding_plane': inputs['slipping_system'][1],
    'slipping_direction': inputs['slipping_system'][2]
}


05/15/2025 04:07:08 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140519|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
05/15/2025 04:07:08 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140519|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
05/15/2025 04:07:08 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140519|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. 
05/15/2025 04:07:08 PM <1288291> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140519|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.0]. 
05/15/2025 04:07:08 PM <1288291> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh

## TEST A2 -- Vanadium

In [4]:

ase_atom = read('./structures/cif/V-227-A2.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([14, 14, 14])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.01
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5



### {100} surface

In [7]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A2', '100', [1/2, 1/2]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/15/2025 12:22:02 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140356|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
05/15/2025 12:22:02 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140356|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
05/15/2025 12:22:02 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140356|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. 
05/15/2025 12:22:02 PM <643100> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140356|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.5]. 
05/15/2025 12:22:02 PM <643100> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh.M

### {110} surface

In [5]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A2', '110', [0.5, 0.5]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/15/2025 01:50:17 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140388|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
05/15/2025 01:50:17 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140388|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
05/15/2025 01:50:17 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140388|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. 
05/15/2025 01:50:17 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140388|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.5]. 
05/15/2025 01:50:17 PM <864510> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh.M

### {111} surface

In [6]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(9)
inputs['slipping_system'] = orm.List(['A2', '111', [1/2, 1/2]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/15/2025 02:15:11 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140418|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #227, prototype V>. I don't know the usual gliding plane. 
05/15/2025 02:15:11 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140418|USFWorkChain|validate_slipping_system]: Now we are working on 111 gliding plane.
05/15/2025 02:15:11 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140418|USFWorkChain|validate_slipping_system]: Faulting is possible for 111 gliding plane.The stacking order is ABC. 
05/15/2025 02:15:12 PM <864510> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140418|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.5]. 
05/15/2025 02:15:12 PM <864510> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoints mesh.

## TEST B1

In [3]:
ase_atom = read('./structures/cif/mpds-S1605628-AsTe-225-B1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([14, 14, 14])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}


inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5


### {100}

In [4]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['B1', '100', [0.5, 0.0]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/16/2025 08:53:31 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140611|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype NaCl>. I don't know the usual gliding plane. 
05/16/2025 08:53:31 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140611|USFWorkChain|validate_slipping_system]: Now we are working on 100 gliding plane.
05/16/2025 08:53:31 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140611|USFWorkChain|validate_slipping_system]: Faulting is possible for 100 gliding plane.The stacking order is AB. 
05/16/2025 08:53:31 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140611|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.5, 0.0]. 
05/16/2025 08:53:32 AM <3275884> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoint

### {110}

In [5]:
inputs['skip_uc'] = orm.Bool(False)
inputs['do_surface_energy'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['B1', '110', [0.0, 0.5]])

logging.getLogger('aiida.transport.SshTransport') .setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

05/16/2025 09:40:47 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140641|USFWorkChain|validate_slipping_system]: FCC element crystal <space group #225, prototype NaCl>. I don't know the usual gliding plane. 
05/16/2025 09:40:47 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140641|USFWorkChain|validate_slipping_system]: Now we are working on 110 gliding plane.
05/16/2025 09:40:47 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140641|USFWorkChain|validate_slipping_system]: Faulting is possible for 110 gliding plane.The stacking order is AB. 
05/16/2025 09:40:47 AM <3275884> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [140641|USFWorkChain|validate_slipping_system]: Slipping direction is manually specified: [0.0, 0.5]. 
05/16/2025 09:40:47 AM <3275884> aiida.workflow.dislocation: [WARNING] kpoints mesh is not compatible with supercell dimensions. Rouding the kpoint

# TEST HALF-HEUSLER

In [11]:

import yaml

ase_atom = read('./structures/cif/mpds-S1612207-NbCoSb-216-C1b.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'minimal'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.01
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5



## {100} surface

In [12]:
inputs['skip_uc'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['C1b', '110', None])

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)

04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|validate_slipping_system]: We are doing half-heusler-type structure, space group (216).
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|validate_slipping_system]: AB stacking for {110} gliding plane. Faulting possible.slipping direction is along 1/2[-111].
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [WARNING] You are skipping the unitcell calculation.Hopefully you know what you are doing!
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|run_USF]: Multiplicity of the faulted geometry: uuid: 5bc1e3a8-792c-4a50-94e7-53ed01b2253d (unstored) value: 8.0
04/26/2025 12:11:01 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|run_USF]